In [1]:
import os

In [3]:
cd ../

/home/asus/Documents/Chest-Cancer-Classification/Project


In [4]:
%pwd

'/home/asus/Documents/Chest-Cancer-Classification/Project'

In [5]:
!pip install dagshub

In [6]:
import dagshub
dagshub.init(repo_owner='Othmen-Derouiche', repo_name='Chest-Cancer-Classification', mlflow=True)


Accessing as Othmen-Derouiche

Initialized MLflow to track repo "Othmen-Derouiche/Chest-Cancer-Classification"

Repository Othmen-Derouiche/Chest-Cancer-Classification initialized!

In [7]:
!pip install ipywidgets

In [8]:
import ipywidgets

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Othmen-Derouiche/Chest-Cancer-Classification.mlflow"



In [9]:
!pip install tensorflow

In [10]:
import tensorflow as tf

2024-08-29 13:05:31.438247: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-29 13:05:31.496165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-29 13:05:31.497265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 13:05:32.488618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

## Update Entity 

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## Update Configuration

In [13]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/Othmen-Derouiche/Chest-Cancer-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

## Update Component 

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG19Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [19]:
import mlflow
import tensorflow as tf

# Patch the mlflow function
def patched_save_model(model, path, conda_env=None, code_paths=None, mlflow_model=None, custom_objects=None,
                       signature=None, input_example=None, pip_requirements=None, extra_pip_requirements=None,
                       saved_model_kwargs=None, keras_model_kwargs=None, metadata=None):
    from mlflow.models import Model
    from mlflow import pyfunc
    from mlflow.utils.environment import _mlflow_conda_env, _REQUIREMENTS_FILE_NAME
    
    if hasattr(tf.keras, '__version__'):
        keras_version = tf.keras.__version__
    else:
        keras_version = tf.__version__

    flavor_options = {
        "keras_version": keras_version,
        "save_format": "h5",
    }

    # rest of the original logic

# Apply the patch
mlflow.tensorflow.save_model = patched_save_model


## Update Pipeline

In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-29 13:05:54,088: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-29 13:05:54,094: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-29 13:05:54,095: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 29s 4s/step - loss: 15.4506 - accuracy: 0.5686
[2024-08-29 13:06:35,940: INFO: common: json file saved at: scores.json]


Registered model 'VGG19Model' already exists. Creating a new version of this model...
2024/08/29 13:06:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG19Model, version 4
Created version '4' of model 'VGG19Model'.
